In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

import tensorflow as tf

In [2]:
df_health = pd.read_csv ('healthcare_dataset.csv')

In [3]:
#add more ages to the column Age with random numbers from 18 to 90.
random_ages=np.random.randint (18,90, size=len (df_health ['Age']))
df_health ['Age'].fillna (pd.Series(random_ages),inplace=True)

In [4]:
#in order to get the NA lower I filled the blkanks with unknown in the field education level
df_health['Education_Level'].fillna('unknown', inplace=True)

In [5]:
print(df_health.columns)


Index(['Patient_ID', 'Age', 'Gender', 'Insurance_Type', 'Chronic_Disease',
       'Mental_Health_Status', 'Employment_Status', 'Education_Level',
       'Transportation_Access', 'Distance_from_Facility', 'Area_Type',
       'Booking_Date', 'Appointment_Date', 'Appointment_Outcome'],
      dtype='object')


In [6]:
# fill the NA of gender by other
unique_gender= df_health['Gender'].unique ()
print(unique_gender)

['Male' 'Other' nan 'Female']


In [7]:
df_health ['Gender'].fillna ('Other',inplace=True)

In [8]:
#change to nominal to ordinal.check the labels
unique_education_levels = df_health['Education_Level'].unique()
print(unique_education_levels)


['Advanced Degree' 'College' 'High School' 'unknown']


In [9]:
ordinal_mapping = {'unknown': 0, 'High School': 1, 'College': 2, 'Advanced Degree': 3}

df_health['Education_Level'] = df_health['Education_Level'].map(ordinal_mapping)

In [10]:
#df_health. head ()
df_health. info ()
#here the count moved from 11000 to 13000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Patient_ID              50000 non-null  int64  
 1   Age                     50000 non-null  float64
 2   Gender                  50000 non-null  object 
 3   Insurance_Type          45009 non-null  object 
 4   Chronic_Disease         45033 non-null  object 
 5   Mental_Health_Status    44962 non-null  object 
 6   Employment_Status       44937 non-null  object 
 7   Education_Level         50000 non-null  int64  
 8   Transportation_Access   45013 non-null  object 
 9   Distance_from_Facility  44983 non-null  float64
 10  Area_Type               45016 non-null  object 
 11  Booking_Date            50000 non-null  object 
 12  Appointment_Date        50000 non-null  object 
 13  Appointment_Outcome     45042 non-null  object 
dtypes: float64(2), int64(2), object(10)
me

In [11]:
df_health = df_health.dropna()

In [12]:
df_health ['Booking_Date']=pd.to_datetime (df_health['Booking_Date'])
df_health ['Appointment_Date']=pd.to_datetime (df_health['Appointment_Date'])
df_health['date_difference']=(df_health ['Appointment_Date']-df_health ['Booking_Date'])
df_health['date_difference'] = df_health['date_difference'].dt.days
df_health['Appointment_Outcome'] = df_health['Appointment_Outcome'].replace({'No-Show': 0, 'Show': 1})
#df_health.head()
df_health. info ()


<class 'pandas.core.frame.DataFrame'>
Index: 21474 entries, 0 to 49998
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Patient_ID              21474 non-null  int64         
 1   Age                     21474 non-null  float64       
 2   Gender                  21474 non-null  object        
 3   Insurance_Type          21474 non-null  object        
 4   Chronic_Disease         21474 non-null  object        
 5   Mental_Health_Status    21474 non-null  object        
 6   Employment_Status       21474 non-null  object        
 7   Education_Level         21474 non-null  int64         
 8   Transportation_Access   21474 non-null  object        
 9   Distance_from_Facility  21474 non-null  float64       
 10  Area_Type               21474 non-null  object        
 11  Booking_Date            21474 non-null  datetime64[ns]
 12  Appointment_Date        21474 non-null  datetime64[

In [13]:
#day of the week lets see if something happens
df_health['day_of_week'] = df_health['Appointment_Date'].dt.day_name()
df_health=pd.get_dummies (df_health,dtype=float)
df_health.head (5)

,Patient_ID,Age,Education_Level,Distance_from_Facility,Booking_Date,Appointment_Date,Appointment_Outcome,date_difference,Gender_Female,Gender_Male,...,Area_Type_Rural,Area_Type_Suburban,Area_Type_Urban,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,75763,93.0,3,37.785038,2023-11-04,2024-08-15,1,285,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,80253,68.0,0,46.619341,2024-08-01,2024-10-22,0,82,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10,94722,76.0,3,21.246094,2023-02-09,2024-04-06,1,422,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11,18645,18.0,3,35.827139,2023-02-14,2024-09-14,1,578,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13,21727,6.0,3,43.314191,2023-12-06,2024-09-12,0,281,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
# Separate features and target variable
X = df_health.drop(['Appointment_Outcome','Appointment_Date','Booking_Date'], axis=1).values
y = df_health['Appointment_Outcome'].values

# X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)



In [11]:
scaler=StandardScaler()
X_scaler=scaler.fit (X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=14, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(units=28, activation='relu'))

# Add Dropout layer for regularization
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model with the legacy Adam optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_scaled, y_train, validation_split=0.15, epochs=100, callbacks=[early_stopping])

# Predictions on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/100
386/386 [==============================] - 0s 542us/step - loss: 0.7166 - accuracy: 0.5028 - val_loss: 0.6964 - val_accuracy: 0.5094
Epoch 2/100
386/386 [==============================] - 0s 427us/step - loss: 0.6971 - accuracy: 0.5019 - val_loss: 0.6946 - val_accuracy: 0.4897
Epoch 3/100
386/386 [==============================] - 0s 421us/step - loss: 0.6938 - accuracy: 0.5125 - val_loss: 0.6944 - val_accuracy: 0.4924
Epoch 4/100
386/386 [==============================] - 0s 410us/step - loss: 0.6922 - accuracy: 0.5145 - val_loss: 0.6944 - val_accuracy: 0.4979
Epoch 5/100
386/386 [==============================] - 0s 405us/step - loss: 0.6919 - accuracy: 0.5199 - val_loss: 0.6943 - val_accuracy: 0.4888
Epoch 6/100
386/386 [==============================] - 0s 415us/step - loss: 0.6915 - accuracy: 0.5252 - val_loss: 0.6943 - val_accuracy: 0.5002
Epoch 7/100
386/386 [==============================] - 0s 413us/step - loss: 0.6915 - accuracy: 0.5224 - val_loss: 0.6939 - val_ac

In [15]:
# Separate features and target variable
X = df_health.drop('Appointment_Outcome', axis=1)
y = df_health['Appointment_Outcome']

# X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# numerical and categorical 
numeric_features = ['Age', 'Distance_from_Facility','Education_Level', 'date_difference']
categorical_features = ['Gender', 'Insurance_Type', 'Mental_Health_Status', 
                         'Employment_Status', 'Transportation_Access', 
                         'Area_Type', 'day_of_week']

# Preprocess numerical features
numeric_transformer = StandardScaler()

# Preprocess categorical features
categorical_transformer = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)]
)

# Combine 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Initialize  within a pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

#  predictions
y_pred = model.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


ValueError: A given column is not a column of the dataframe

In [8]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.5017035775127768
Confusion Matrix:
 [[615 560]
 [610 563]]
Classification Report:
               precision    recall  f1-score   support

     No-Show       0.50      0.52      0.51      1175
        Show       0.50      0.48      0.49      1173

    accuracy                           0.50      2348
   macro avg       0.50      0.50      0.50      2348
weighted avg       0.50      0.50      0.50      2348



In [9]:
print(df_health.columns)


Index(['Patient_ID', 'Age', 'Gender', 'Insurance_Type', 'Chronic_Disease',
       'Mental_Health_Status', 'Employment_Status', 'Education_Level',
       'Transportation_Access', 'Distance_from_Facility', 'Area_Type',
       'Booking_Date', 'Appointment_Date', 'Appointment_Outcome',
       'date_difference', 'day_of_week'],
      dtype='object')


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


# One-hot encode categorical variables using pd.get_dummies
categorical_columns = ['Gender', 'Insurance_Type', 'Mental_Health_Status', 
                       'Employment_Status', 'Education_Level', 
                       'Transportation_Access', 'Area_Type',]

df_health = pd.get_dummies(df_health, columns=categorical_columns, drop_first=True)

# Separate features and target variable
X = df_health.drop('Appointment_Outcome', axis=1, errors='ignore')
#X = df_health.drop(['Appointment_Outcome', 'day_of_week', 'Appointment_Date'], axis=1)
#X = df_health.drop('Appointment_Outcome', axis=1)
y = df_health['Appointment_Outcome']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Preprocess numerical features
numeric_transformer = StandardScaler()

# Initialize within a pipeline
model = Pipeline(steps=[
    ('scaler', numeric_transformer),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

TypeError: Cannot cast DatetimeArray to dtype float64